# Converting BDTNS dates to Processed Dates

The purpose of this notebook is to take the dates from the Drehem dataset and convert to dates that are easier to work with. The 0th date in the processed dates is the beginning of Ur-Namma's period. 

Notebook by Tiffany Chien for the Sumerian Network project.

# 1. Getting text file from BDTNS

In order to get P numbers and dates, go to 
http://bdtns.filol.csic.es/ 

In the Catalogue & Transliteration drop down menu choose "Search" 

In the Search form go to Provenience (another drop down menu) and choose Puzrish-Dagan (the ancient name of Drehem) and hit "Search". This will yield a long list of all tablets from Drehem - some 15,000 of them. In the left pane there is an Export button. Once you hit the button you get a number of options. Select "CDLI number" (= P number) and Date and hit "export". This will produce a .txt file that you can manipulate further.


Importing information from text file as a dataframe

In [ ]:
import pandas as pd
import glob
import os
list_of_files = glob.glob('bdtns_metadata/*')
list_of_files = [f for f in list_of_files if 'query_cat_' in f]
list_of_files.sort()
TEXT_FILE_PATH = list_of_files[0]
drehem_date_data = pd.read_csv(TEXT_FILE_PATH, delimiter='\t', header=None)

Column 1 contains the CDLI numbers and Column 2 contains the unprocessed dates.

In [ ]:
drehem_date_data.head(7)

#  2. Removing rows with null dates

In [ ]:
dates = drehem_date_data[2]
drehem_date_data = drehem_date_data[~dates.isna()]

Mapping KINGS to years. We subtract the years from the bias as we want processed dates to start from the beginning of Ur-Namma's reigning period as 0. 

The names mapping is not used later in the code but is there to show which abbreviation stands for whom. 

In [ ]:
BIAS = 2112

KINGS = {
	'Ur-Namma': (BIAS - 2112, BIAS - 2095),
	'SH': (BIAS - 2094, BIAS - 2047),
	'AS': (BIAS - 2046, BIAS - 2038),
	'SS': (BIAS - 2037, BIAS - 2029),
	'IS': (BIAS - 2028, BIAS - 2004)
}


names = {'IS': 'Ibbi-Suen',
         'SS': 'Shu-Suen',
         'AS': 'Amar-Suen',
         'SH': 'Shulgi',
        }

# 3. Converting months and days to appropriate decimal representation

If any part of the dates has an invalid value, we represent it as 0 in this dataset. The catch_num_formatting function converts a number to its appropriate value for adding to the total date.

For example, **catch_num_formatting(07, '0.')** would return 0.07. This value would then be added to the total date as month. 

To represent month, we call **catch_num_formatting(month, '0.')**.<br />
To represent a day, we call **catch_num_formatting(day, '0.00')**.

Note: Could also choose to exclude it from entire dataset if needed

In [ ]:
INVALID_DATES = {'00', '--', '--,', 'XX', '??'}

def catch_num_formatting(number, decimal):
    if number in INVALID_DATES or 'x' in number:
        return 0
    try:
        return float(decimal + number)
    except ValueError:
        print('num formatting', number)
        return 0

Here is an example of converting a singular date to a proccessed date. Splitting the date by '-' and stripping them of white spaces seperates the year, month, and day into 3 strings. 

The processed date is the summation of the year, the converted month, and the converted date. The first two characters of the string contain information about the King(retrieved from the KINGS mapping) and the last two characters specify the amount of years into the king's period. 

In [ ]:
a_date = drehem_date_data[2][0].split('-')
print(a_date)
year = a_date[0].strip()
month = a_date[1].strip()
day = a_date[2].strip()


processed = KINGS[year[0:2]][0]+int(year[2:]) + catch_num_formatting(month, '0.') + catch_num_formatting(day, '0.00')
print('processed',processed)

Processing all dates in dataframe from txt file

In [ ]:
def process_dates(a_date):
    try:
        if a_date not in INVALID_DATES:
            split_date = a_date.split('-')
            year = process_year(split_date[0])
            month = split_date[1].strip()[0:2]
            day = split_date[2].strip()[0:2]
            processed = year + catch_num_formatting(month, '0.') + catch_num_formatting(day, '0.00')
            return processed
    except:
        print('This date caused an error: ',a_date)
        return 0

def process_year(year):
    try:
        if year[0:2] in INVALID_DATES:
            return 0
        else:
            return KINGS[year[0:2]][0]+int(year[2:])
    except:
        return 0
        
drehem_date_data[3] = drehem_date_data[2].apply(process_dates)
drehem_date_data[drehem_date_data[2]!=0].head()


In [ ]:
#adding column names
drehem_date_data.rename(columns={0: 'BDTNS No', 1: 'CDLI No', 2: 'Original date', 3:'Converted Date'}, inplace = True)
drehem_date_data.head(3)

# 4. Writing to CSV

In [ ]:
drehem_date_data.to_csv("bdtns_metadata/dates.csv")

In [ ]:
#outlier
#removed from dataset for visualization purposes

print('Removed the following rows in visualization:')
print(drehem_date_data[drehem_date_data['Converted Date']>400])
drehem_date_data = drehem_date_data[drehem_date_data['Converted Date']<400]



In [ ]:
ax = drehem_date_data.plot.scatter(x=0, 
                                   y = 'Converted Date',)